In [5]:
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()


In [6]:
len(text)

1115393

In [7]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)


In [11]:
n = int(0.9 * len(text))
train_data = data[:n]
val_data = data[n:]

In [12]:
torch.manual_seed(1337)

block_size = 8
batch_size = 4
train_data[:block_size + 1]

def get_batch(split):
    data = train_data if split == "train" else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

xb, yb = get_batch("train")

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            # we're just predicting single token
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # note no targets
            logits, loss = self(idx)
            # just pluck out the last prediction in T dimension
            # (B, T, C) -> (B, C)
            logits = logits[:, -1, :]

            # calculate softmax for the results for the inputs
            probs = F.softmax(logits, dim=-1)

            # 1 sample
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # becomes (, T+1)
            idx = torch.cat((idx, idx_next), dim=-1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

NEWLINE_IDX = 0
idx = torch.tensor(([[NEWLINE_IDX]]), dtype=torch.long)

output = m.generate(idx, max_new_tokens=100)[0] # because first batch
output = output.tolist() # convert from tensor to list
output = decode(output) # to string
print(output)



torch.Size([32, 65])
tensor(4.9456, grad_fn=<NllLossBackward0>)

lfJeukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH


In [14]:
# training the bigram model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.6492578983306885


In [16]:
# try generating again
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Ufe hoszysqPUg;ly3pueZL bfNUpr $CftcukI$O$VNotpklckNbbfJQ:
EN3MpHe,'DPUIRRA$;URELopLY?Ancorte yXDl?o'WtKB:ur 
ENJ zkAQB$ZPzw.K,XrMnroCOda,fd:HJoeNo:IRSl;toiY!LY:nsnG?WESqXYGJmr,jzmO,o w ;in:; :Y bexW,l;!OMlUzurdZLCiwW3bbATq$waifvjo QAYerBigart::uotpTh
N3vdoewW3g;k
Ut?O?viw ifJs
is:HENUCl;eo Cyd zEzb


In [17]:
torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
xbow = torch.zeros((B, T, C))

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # up to and incluidng the th token
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
#version 2

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (_B, T, T) @ (B, T, C) 
torch.allclose(xbow, xbow2)


True

In [20]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [21]:
# version 4: self-attention
B, T,  C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)

out = wei @ v # (B, T, T) @ (B, T, 16) -> (B, T, 16)
out.shape

torch.Size([4, 8, 16])

In [30]:
class BatchNorm1d:
    # assume affine True (will be using gamma and beta)
    # track running stats is true
    def __init__(self, dim: int, eps=1e-5, momentum=0.1):
        # the bngain and bnbias
        self.eps = eps
        self.gamma = torch.ones(dim)  # initially scale as 1
        self.beta = torch.zeros(dim)  # initially no shift

    def __call__(self, x):
        # forward pass
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)

        # not in pytorch - just to make it easier to calculate statistics
        self.out = self.gamma * xhat + self.beta

        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape


torch.Size([32, 100])

In [27]:
x[0, :].mean(), x[0, :].var()


(tensor(2.3842e-09), tensor(1.0000))